<a href="https://colab.research.google.com/github/jeff-ai-ml/silver-badge-project/blob/main/Assignment_1_Car_Consumer_Report_2024_Analysis_RAG_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# === Install Required Packages ===

In [1]:
!pip install langchain-community langchain-mistralai pypdf chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.8 MB/s eta 0:0

In [2]:
# Import libraries
import os
from langchain_community.document_loaders import PyPDFLoader #For reading and parsing PDF
from langchain.text_splitter import RecursiveCharacterTextSplitter # For spliting characters and making chunks of parsed PDF content
from langchain_community.embeddings import HuggingFaceEmbeddings # For creating embedding of chunks of characters
from langchain_community.vectorstores import Chroma # For storing the embedding in vector database
from langchain.prompts import ChatPromptTemplate # For creating Prompt to RAG
from langchain_mistralai import ChatMistralAI # For Mistral AI API

# === Set Paths ===

In [3]:
CHROMA_PATH = "Chroma"
DOC_PATH = "/content/sample_data/Top-Car-Picks-for-2024-by-Consumer-Reports-PDF.pdf"

# === Step 1: Load and Chunk PDF ===

In [14]:
# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()
print(type(pages))
print(len(pages))
print(pages)

<class 'list'>
8
[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2024-03-05T19:39:10+00:00', 'title': 'Top Car Picks for 2024 by Consumer Reports', 'author': 'Tony Rached', 'subject': 'Top Car Picks for 2024 by Consumer Reports', 'keywords': 'Top Car Picks for 2024 by Consumer Reports', 'moddate': '2024-03-05T19:39:10+00:00', 'source': '/content/sample_data/Top-Car-Picks-for-2024-by-Consumer-Reports-PDF.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='https://diminishedvaluecarolina.\ncom \n \n \n \n \nTop Car Picks for \n2024 by Consumer \nReports'), Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2024-03-05T19:39:10+00:00', 'title': 'Top Car Picks for 2024 by Consumer Reports', 'author': 'Tony Rached', 'subject': 'Top Car Picks for 2024 by Consumer Reports', 'keywords': 'Top Car Picks for 2024 by Consumer Reports', 'moddate': '2024-03-05T19:39:10+00:00', 'source': '/content/sample_data/To

In [16]:
# split the doc into smaller chunks i.e. chunk_size=500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)
print(type(chunks))
print(len(chunks))
print(chunks)

<class 'list'>
18
[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2024-03-05T19:39:10+00:00', 'title': 'Top Car Picks for 2024 by Consumer Reports', 'author': 'Tony Rached', 'subject': 'Top Car Picks for 2024 by Consumer Reports', 'keywords': 'Top Car Picks for 2024 by Consumer Reports', 'moddate': '2024-03-05T19:39:10+00:00', 'source': '/content/sample_data/Top-Car-Picks-for-2024-by-Consumer-Reports-PDF.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='https://diminishedvaluecarolina.\ncom \n \n \n \n \nTop Car Picks for \n2024 by Consumer \nReports'), Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2024-03-05T19:39:10+00:00', 'title': 'Top Car Picks for 2024 by Consumer Reports', 'author': 'Tony Rached', 'subject': 'Top Car Picks for 2024 by Consumer Reports', 'keywords': 'Top Car Picks for 2024 by Consumer Reports', 'moddate': '2024-03-05T19:39:10+00:00', 'source': '/content/sample_data/T

# === Step 2: Embeddings (HuggingFace) ===

In [17]:
# get Embedding model (keep as is, as you specified no change to vector DB)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-17-2601280204.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# === Step 3: Create Embeddings and store in ChromaDB (Vector Database) ===

In [19]:
# embed the chunks as vectors and load them into the database.
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)

# === Step 4: Define User Query ===

In [26]:
# this is an example of a user question (query)
query = 'according to consumer report 2024 which is the top car brand?'

# === Step 5: Retrieve Similar Chunks ===

In [27]:
# retrieve context - top 5 most relevant (closest) chunks to the query vector
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)

In [28]:
#see the content of similarity search
docs_chroma

[(Document(metadata={'producer': 'PyPDF', 'total_pages': 8, 'author': 'Tony Rached', 'keywords': 'Top Car Picks for 2024 by Consumer Reports', 'source': '/content/sample_data/Top-Car-Picks-for-2024-by-Consumer-Reports-PDF.pdf', 'moddate': '2024-03-05T19:39:10+00:00', 'creationdate': '2024-03-05T19:39:10+00:00', 'creator': 'Microsoft Word', 'title': 'Top Car Picks for 2024 by Consumer Reports', 'page_label': '5', 'subject': 'Top Car Picks for 2024 by Consumer Reports', 'page': 4}, page_content='the future of driving is electric, efficient, and exciting. \nNow, let’s dive into the specifics with the brand report card and top picks by category \nto provide a comprehensive look at what makes these vehicles stand out: \n2024 Brand Report Card \nConsumer Reports ranked 34 brands based on the overall scores of their models, \nwhich include road tests, reliability, satisfaction, and safety. \nRank Brand Score \n1 BMW 82 \n2 Subaru 80 \n3 Porsche 80 \n4 Honda 78 \n5 Lexus 78 \n6 Mini 78 \n7 Kia

In [33]:
# generate an answer based on given user query and retrieved context information
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

In [34]:
# see the content of context_text
context_text

"the future of driving is electric, efficient, and exciting. \nNow, let’s dive into the specifics with the brand report card and top picks by category \nto provide a comprehensive look at what makes these vehicles stand out: \n2024 Brand Report Card \nConsumer Reports ranked 34 brands based on the overall scores of their models, \nwhich include road tests, reliability, satisfaction, and safety. \nRank Brand Score \n1 BMW 82 \n2 Subaru 80 \n3 Porsche 80 \n4 Honda 78 \n5 Lexus 78 \n6 Mini 78 \n7 Kia 77\n\nhttps://diminishedvaluecarolina.\ncom \n \n2 \n \nSafety and Satisfaction: Non-Negotiables \nWhat sets the 2024 Top Picks apart is not just their innovative powertrains but their \nuncompromising commitment to safety and owner satisfaction. \nEach vehicle on the list comes standard with key safety features, reflecting Consumer \nReports' dedication to ensuring that the best cars are also the safest. \nAdditionally, high marks for reliability and satisfaction from CR's member surveys\n\n

# === Step 6: Prompt Template ===

In [29]:
# you can use a prompt template
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

# === Step 7 - Set Mistral as LLM for RAG ===

In [30]:
# Retrieve the Mistral API key from Colab Secrets
import os
from google.colab import userdata

try:
    # 1. Retrieve the secret from Colab's userdata.
    MISTRAL_API_KEY = userdata.get("MISTRAL_API_KEY")
    # 2. Set it as an environment variable for the current session.
    os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY
except Exception as e:
    print(f"Error retrieving MISTRAL_API_KEY from Colab Secrets: {e}")
    print("Please ensure you have set 'MISTRAL_API_KEY' in Colab Secrets and granted notebook access.")
    exit() # Stop execution if the key isn't found, as the LLM won't work.

# === Step 8 -  Initialize Mistral LLM ===

In [31]:
# Mistral AI's official API:
# Can choose "mistral-tiny", "mistral-small", "mistral-medium"
model = ChatMistralAI(model="mistral-small", temperature=0.7)

In [35]:
# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

In [36]:
# call LLM model to generate the answer based on the given context and query
response_text = model.invoke(prompt) # Use .invoke() for ChatModels in Langchain
print(response_text.content) # Access the content of the response

The top car brand according to the Consumer Reports 2024 ranking is BMW, with a brand score of 82 out of 100. The ranking is based on the overall scores of the brands' models, including road tests, reliability, satisfaction, and safety.
